### Libraries

In [1]:
# import libraries

import cv2
import os
import json

### Data directory

In [2]:
# main paths

images_dir = './samples'
input = 'input.json'
output = 'output.json'

### Input file

In [47]:
# open an image

with open(input, 'r') as json_file:
    data = json.load(json_file)

image_paths = data['image_files']
image_index = 7

original_image = cv2.imread(os.path.join(images_dir, image_paths[image_index]))
original_image = cv2.resize(original_image, (0, 0), fx = 0.15, fy = 0.15)

cv2.imshow('Original', original_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [48]:
# gray and hsv versions

gray_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

hsv_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2HSV)

cv2.imshow('Gray', gray_image)
cv2.imshow('HSV', hsv_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Lego detection

In [50]:
image = original_image.copy()
gray = gray_image.copy()

clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
image_CLAHE = clahe.apply(gray)

gaussian = cv2.GaussianBlur(image_CLAHE, (7, 7), 0)

canny = cv2.Canny(gaussian, 0, 300)

contours, _ = cv2.findContours(canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)

num_lego_pieces = len(contours)

cv2.imshow('Image', image)
cv2.imshow('CLAHE', image_CLAHE)
cv2.imshow('Gaussian', gaussian)
cv2.imshow('Canny', canny)
cv2.waitKey(0)
cv2.destroyAllWindows()

print("Number of LEGO pieces detected:", num_lego_pieces)

Number of LEGO pieces detected: 2
